# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_AI_KET')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')


# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())

# OPENAI_API_KEY  = os.getenv('OPENAI_AI_KEY')
# HUGGINGFACEHUB_API_TOKEN = os.getenv('HF_TOKEN')

In [5]:
!pip install pandas

In [9]:
import pandas as pd
df = pd.read_csv('/Data.csv')

In [10]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [11]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [13]:
!pip install langchain-openai
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 2.6 MB/s eta 0:00:00


In [27]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.9)
#temperature param determines the randomness of the output. The higher the temperature, the more unique and creative the output is.

In [18]:
prompt = ChatPromptTemplate.from_template("This program will generate a review of any product")
    #Write a query that would take a variable to describe any product


In [28]:

chain = LLMChain(llm=llm, prompt=prompt)

In [30]:
product = "lawn ornaments"
chain.run(input=product)

'Please provide the name of the product you would like a review for.'

## SimpleSequentialChain

In [31]:
from langchain.chains import SimpleSequentialChain

In [37]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template("Generate a review for the movie: {movie_title}"
    #Repeat the initial query or create a new query that would feed into the second prompt
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [39]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template("Write a summary of the following movie review: {review}")
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [40]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [41]:
movie_title = "Inception"

overall_simple_chain.run(movie_title)



> Entering new SimpleSequentialChain chain...
Inception is a mind-bending masterpiece that will leave you questioning reality long after the credits roll. Directed by the visionary Christopher Nolan, this film is a true cinematic experience that challenges your perceptions of dreams and consciousness. The film's intricate plot, stunning visual effects, and thought-provoking themes are expertly woven together to create a suspenseful and immersive experience. The stellar performances from the cast, particularly Leonardo DiCaprio and Joseph Gordon-Levitt, elevate the film to new heights. Inception is a truly unique and unforgettable film that will captivate and intrigue audiences for years to come.
The reviewer praises Christopher Nolan's "Inception" as a mind-bending masterpiece that challenges perceptions of dreams and consciousness. The film's intricate plot, stunning visual effects, and thought-provoking themes create a suspenseful and immersive experience that will leave viewers qu

'The reviewer praises Christopher Nolan\'s "Inception" as a mind-bending masterpiece that challenges perceptions of dreams and consciousness. The film\'s intricate plot, stunning visual effects, and thought-provoking themes create a suspenseful and immersive experience that will leave viewers questioning reality long after the credits roll. Stellar performances from Leonardo DiCaprio and Joseph Gordon-Levitt elevate the film to new heights, making it a truly unique and unforgettable cinematic experience that will captivate and intrigue audiences for years to come.'

**Repeat the above twice for different products**

## SequentialChain

In [42]:
from langchain.chains import SequentialChain

In [44]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Generate a translation for a review: {review}"
   #This prompt should translate a review
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="arabic_text" #Give a name to your output
                    )

In [45]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a summary of the following review: {arabic_text}"
    #Write a promplt to summarize a review
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="arabic_review" #give a name to this output
                    )


In [46]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english: {arabic_review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="english_text"
                      )


In [47]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Generate a thank you reading message in {arabic_text} and {english_text}"

)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="thank_you_message"
                     )


In [49]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["arabic_text", "english_text", "thank_you_message"],
    verbose=True
)

In [50]:
review = df.Review[5]
overall_chain(review)

<ipython-input-50-1992003631>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'arabic_text': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'english_text': 'The reviewer found the taste of the product to be mediocre and noted that the foam did not hold well, which they found strange. They mentioned that they typically buy the same product in stores and find the taste to be much better, leading them to question if the product they received was from an old batch or if it may have been counterfeit.',
 'thank_you_message': 'Thank you for taking the time to provide feedback on your recent purchase. We are sorry to hear that you found the taste of the product to be mediocre and that the foam did not hold well. We take your comments seriously and will look into the possibility of wheth

**Repeat the above twice for different products or reviews**

## Router Chain

In [51]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [52]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [53]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [54]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [55]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [56]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [57]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [58]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

<ipython-input-58-3038952769>:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [59]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation incident on it and does not reflect any. This radiation has a characteristic spectrum that depends only on the body's temperature, and follows Planck's law. It is often used as a theoretical model for idealized physical systems and plays a crucial role in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [60]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 equals 4.'

In [61]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for how our bodies function and develop. DNA contains the necessary information for building and maintaining all the proteins and structures that make up our bodies. It also holds the genetic code that is passed on from one generation to the next, ensuring that traits and characteristics are inherited.\n\nThe presence of DNA in every cell allows for proper growth, development, and functioning of the body. It serves as a blueprint for the production of specific proteins that carry out essential functions, such as enzymes for metabolism, structural proteins for support, and signaling molecules for communication between cells. Without DNA, cells would not be able to carry out their specific functions and the body would not be able to survive.\n\nAdditionally, having DNA in every cell ensures that genetic information is preserved and transmitted accurately during cell division. This is cr

**Repeat the above at least once for different inputs and chains executions - Be creative!**